In [2]:
# text classifier using classy classification
# Didn't work properly
 
import spacy
import classy_classification
import os
directory = 'apply'

# List all files and directories in the specified directory
all_files = os.listdir(directory)
content=""
for file in all_files[:round(len(all_files)/2)]:
    with open(os.path.join(directory,file), 'r') as f:
        partial_content = f.read()
        partial_content = partial_content.replace('\n','.' )
    content+=partial_content
content

'18e590724fa72a8f.Workday - Accolade <osv_accolade@myworkday.com>.Thank you for applying to Accolade, Inc..Hi Kelvin , Thank you very much for your interest in Accolade and the position of Intern - Software Engineer (Python, Spark) . Your resume will be reviewed by one of our recruiters shortly and, if your background experience meets the criteria for the position, we will be in touch to schedule an interview with a member of the Talent Acquisition team via email or phone. If you have any questions, please feel free to reach out to any member of the recruiting team for additional information. Sincerely, Accolade Talent Acquisition Team This email was intended for kelvin4jaison@gmail.com Accolade · 660 W Germantown Pike Suite 500 Plymouth Meeting, PA 19460.18e492a19f4eab0e.AMETEK Talent Acquisition Team <AMETEK.Recruiting@successfactors.com>.Thank you for applying to Ametek, Inc..Dear Kelvin, Your application for the position of \u200bZygo - Software Engineering Intern has been received

In [3]:
#creating data dictionary
data = {}
justSentence=spacy.blank("en")
justSentence.add_pipe("sentencizer")
apply_sentence=justSentence(content).sents
data["apply"]=[sent.text for sent in apply_sentence]


In [4]:
with open('rejectPatterns.txt', 'r') as file:
    content = file.read()
    content = content.replace('\n','.' )
reject_sentence=justSentence(content).sents
data["reject"]=[sent.text for sent in reject_sentence]


In [5]:
data

{'apply': ['18e590724fa72a8f.',
  'Workday - Accolade <osv_accolade@myworkday.com>.Thank you for applying to Accolade, Inc..Hi Kelvin , Thank you very much for your interest in Accolade and the position of Intern - Software Engineer (Python, Spark) .',
  'Your resume will be reviewed by one of our recruiters shortly and, if your background experience meets the criteria for the position, we will be in touch to schedule an interview with a member of the Talent Acquisition team via email or phone.',
  'If you have any questions, please feel free to reach out to any member of the recruiting team for additional information.',
  'Sincerely, Accolade Talent Acquisition Team This email was intended for kelvin4jaison@gmail.com Accolade · 660 W Germantown Pike Suite 500 Plymouth Meeting, PA 19460.18e492a19f4eab0e.',
  'AMETEK Talent Acquisition Team <AMETEK.Recruiting@successfactors.com>.Thank you for applying to Ametek, Inc..Dear Kelvin, Your application for the position of \u200bZygo - Softwar

In [6]:
#creating model
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe(
    "classy_classification",
    config={
        "data": data,
        "model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    }
)


In [7]:
#trial with a file
file='reject/reject16.txt'
with open(file, 'r') as file:
    content = file.read()
    content = content.replace('\n',' ' )
test_sentences=justSentence(content).sents
test_sentences=[sent.text for sent in test_sentences]
#print(nlp("After reviewing your resume, we have decided to move forward with other candidates at this time.")._.cats)

for sent in test_sentences:
    doc=nlp(sent)
    threshold=0.5
    if doc._.cats['apply']>threshold or doc._.cats['reject']>threshold:
        print(sent,doc._.cats)

18e5ac1b55e6dfbc noreply epri <vhr_epri@myworkday.com> Thank you for your interest in the Electric Power Research Institute Dear Kelvin Konnoth, Thank you for submitting your resume to the Electric Power Research Institute in consideration for REQ-3235 Nuclear Software Developer Student Employee (Open) . {'apply': 0.9990196805126844, 'reject': 0.0009803194873156168}
We were fortunate to have many qualified candidates apply to each of our positions. {'apply': 0.8110043569314227, 'reject': 0.18899564306857725}
We have reviewed the qualifications of each candidate and after careful consideration; we have determined that the credentials of other candidates may better fit our needs at this time. {'apply': 0.8400095460497022, 'reject': 0.15999045395029768}
We will keep your resume on hand for future consideration and encourage you to express further interest in positions you feel may be a strong fit for your background and experience. {'apply': 0.9882395054438322, 'reject': 0.011760494556167

In [8]:
# testing all apply
import os
directory = 'apply'
all_files = os.listdir(directory)
for file in all_files:
    with open(os.path.join(directory,file), 'r') as file:
        content = file.read()
        content = content.replace('\n',' ' )
    test_sentences=justSentence(content).sents
    test_sentences=[sent.text for sent in test_sentences]
    for sent in test_sentences:
        doc=nlp(sent)
        threshold=0.3
        if doc._.cats['reject']>threshold:
            print(sent,doc._.cats)


In [9]:
#testing all reject
import os
directory = 'reject'
all_files = os.listdir(directory)
for file in all_files:
    with open(os.path.join(directory,file), 'r') as f:
        content = f.read()
        content = content.replace('\n',' ' )
    test_sentences=justSentence(content).sents
    test_sentences=[sent.text for sent in test_sentences]
    print(os.path.join(directory,file))
    for sent in test_sentences:
        doc=nlp(sent)
        threshold=0.8
        if doc._.cats['reject']>threshold:
            print(sent,doc._.cats)


reject/reject11.txt
At this time, however, we are unable to offer you an interview for the Software Engineer - Data Architecture Intern role. {'apply': 0.04118414297687385, 'reject': 0.9588158570231263}
reject/reject10.txt
We’re fortunate to have received a lot of interest in this role, resulting in a very competitive selection process and unfortunately, at this time we have decided not to move forward with your application. {'apply': 0.005603116146731587, 'reject': 0.9943968838532682}
reject/reject12.txt
We’re fortunate to have received a lot of interest in this role, resulting in a very competitive selection process and unfortunately, at this time we have decided not to move forward with your application. {'apply': 0.005603116146731587, 'reject': 0.9943968838532682}
reject/reject13.txt
After careful consideration, we regret to inform you that you have not been selected for this position. {'apply': 0.15657614482065632, 'reject': 0.8434238551793437}
reject/reject17.txt
reject/reject16.